<a href="https://colab.research.google.com/github/robertoarturomc/ProgramacionConcurrente/blob/main/14_Multithreading_en_Python_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Programación Concurrente
## 14. Multithreading en Python II

### El Algoritmo *Quick sort* para ordenar datos.

![Quick sort2](https://cdn-images-1.medium.com/max/1080/1*zI6AMtMYGWqOQUo80r3v8A.jpeg)



1. Elegir un *Pivote*. Normalmente, agarraremos el último o el primer valor de mi arreglo.
2. Colocar los demás valores de ambos lados del pivote. Los más chicos, a la izquierda y los más grandes, a la derecha. Los valores iguales al pivote, pueden ir de cualquier lado.
3. Repetir de manera recursiva las operaciones con cada una de las mitades.

La implementación tiene complejidad $O(n*log n)$, más baja que otros algoritmos.

Si se fijan, una vez dividido el arreglo, ambos se van ordenando de manera independiente. ¿Será posible, entonces, tener una implementación Concurrente?

In [5]:
import threading

In [6]:
# Voy a crear una función que me ayude con la partición
def partition(nums, low, high):
# i va a ser la posición donde va a quedar mi elemento pivote
    i = low - 1
    pivot = nums[high]

    # Hago la comparativa valor a valor
    for j in range(low, high):
        if nums[j] <= pivot:
            i = i + 1
            nums[i], nums[j] = nums[j], nums[i]

    nums[i + 1], nums[high] = nums[high], nums[i + 1]
    return i + 1


In [4]:
def quick_sort(nums, low, high):
    if low < high:
        pi = partition(nums, low, high)

        # Creo dos threads, uno para cada lado de la función
        left_thread = threading.Thread(target=quick_sort, args=(nums, low, pi - 1))
        right_thread = threading.Thread(target=quick_sort, args=(nums, pi + 1, high))

        # Inicializo ambos hilos
        left_thread.start()
        right_thread.start()

        # Finalizo mis hilos
        left_thread.join()
        right_thread.join()


# Ejemplo
arr = [4, 5, 8, 3, 0, 5, 3, 9, 4, 3]
print("Arreglo original:", arr)

# Ordeno mediante multihilos
quick_sort(arr, 0, len(arr) - 1)

print("Arreglo Ordenado:", arr)

Arreglo original: [4, 5, 8, 3, 0, 5, 3, 9, 4, 3]
Arreglo Ordenado: [0, 3, 3, 3, 4, 4, 5, 5, 8, 9]


No obstante, como ya lo hemos platicado antes, usar Concurrencia multihilo no es útil en todas las aplicaciones. El avance, finalmente, sólo se va alternando entre ambas tareas, por lo que el tiempo al final suele ser bastante similar a un equivalente Secuencial.

Pero la principal aplicación del multihilo es durante las tareas de I/O (Input/Output), donde la CPU permanece inactiva mientras espera que se carguen los datos. El multihilo juega un papel crucial aquí, ya que este tiempo de inactividad de la CPU se utiliza en otras tareas, lo que lo hace ideal para la optimización.


### Requests en internet

Para este ejemplo, vamos a usar una de tantas librerías que permiten descargar información. Si lo hacemos de manera Concurrente, cada una se inicializa y comienza una descarga independiente. En este caso, el tiempo de respuesta sí será más veloz que hacerlo de manera Secuencial.

In [12]:
import urllib.request


# Defino mi función
def download_file(url, filename):
    print(f"\Descargando {filename} from {url}...")
    urllib.request.urlretrieve(url, filename)
    print(f"\n{filename} descarga exitosa.")

# ¿Qué vamos a descargar?
files_to_download = [
    {"url": "https://en.wikipedia.org/wiki/British_logistics_in_the_Normandy_campaign", "filename": "i:\wfile1.html"},
    {"url": "https://en.wikipedia.org/wiki/Graph_(abstract_data_type)", "filename": "i:\Graph_abstract_data_type.html"},
    {"url": "https://pbs.twimg.com/profile_images/1237395014071640067/M5dHDl-g_400x400.jpg", "filename": "i:\example.jpg"}
]

# Creo una lista donde iré almacenando mi hilos
threads = []

# Creo un hilo por cada archivo y comienzo mi descarga
for file_info in files_to_download:
    thread = threading.Thread(
        target=download_file,
        args=(file_info["url"], file_info["filename"])
    )
    thread.start()
    threads.append(thread)

# Me espero a terminar mis descargas
for thread in threads:
    thread.join()

\Descargando i:\wfile1.html from https://en.wikipedia.org/wiki/British_logistics_in_the_Normandy_campaign...
\Descargando i:\Graph_abstract_data_type.html from https://en.wikipedia.org/wiki/Graph_(abstract_data_type)...
\Descargando i:\example.jpg from https://pbs.twimg.com/profile_images/1237395014071640067/M5dHDl-g_400x400.jpg...

i:\example.jpg descarga exitosa.

i:\Graph_abstract_data_type.html descarga exitosa.

i:\wfile1.html descarga exitosa.


## Tarea
Invesigar cómo funciona el paquete `multiprocessing` en Python